In [1]:
import pandas as pd
from sklearn import metrics
import numpy as np


trainBrut = pd.read_csv("train.csv")

train = trainBrut.drop(columns=['Name', 'Cabin', 'Ticket'])

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

featuresToDedisc = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

train['hyp'] = 0

testBrut = pd.read_csv("test.csv")
test = testBrut.drop(columns=['Name', 'Cabin', 'Ticket'])
test['Survived'] = 0

In [2]:
def stats2(test):

    print(metrics.classification_report(test.Survived, test.hyp, target_names = ['Dead', 'Alive'], zero_division=1))


In [32]:
def distMan(dataX, dataY, featuresNames):

    dist = 0
    for f in featuresNames:

        if f in featuresToDedisc:
            if dataX[f] != dataY[f]:
                dist += 1
        else:
            dist += np.abs(dataY[f] - dataX[f])

    return dist

In [35]:
from heapq import nsmallest

def KPPV(test, train, featuresNames, k):

    for testInd in test.index:
        distances = {}
        for trainInd in train.index:
            distances[trainInd] = distMan(test.iloc[testInd], train.iloc[trainInd], featuresNames)
        
        closestIndex = nsmallest(k, distances, key = distances.get)
        
        closestClasses = []

        for trainInd in closestIndex:
            closestClasses.append(train.iloc[trainInd].Survived)
        test.hyp[testInd] = max(closestClasses,key=closestClasses.count)

In [38]:
KPPV(train, train, features, 4)

test


C:\Users\romai\AppData\Local\Temp\ipykernel_8512\2334540847.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Survived[testInd] = max(closestClasses,key=closestClasses.count)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,3,male,34.5,0,0,7.8292,Q,0
1,893,3,female,47.0,1,0,7.0000,S,0
2,894,2,male,62.0,0,0,9.6875,Q,1
3,895,3,male,27.0,0,0,8.6625,S,1
4,896,3,female,22.0,1,1,12.2875,S,0
...,...,...,...,...,...,...,...,...,...
413,1305,3,male,NaN,0,0,8.0500,S,1
414,1306,1,female,39.0,0,0,108.9000,C,1
415,1307,3,male,38.5,0,0,7.2500,S,0
416,1308,3,male,NaN,0,0,8.0500,S,1


In [37]:
test.to_csv('res.csv', columns=['PassengerId', 'Survived'])

In [39]:
stats2(train)

              precision    recall  f1-score   support

        Dead       0.50      1.00      0.67       448
       Alive       1.00      0.00      0.00       443

    accuracy                           0.50       891
   macro avg       0.75      0.50      0.33       891
weighted avg       0.75      0.50      0.34       891

